In [623]:
# Домашняя работа к занятию №3. От команды №3
#Состав команды:
#Александра Малинина Amalinina
#Елена Зарубина Elena Zarubina
#Александр Иванин aliv65
#Александр Халипов
#Дмитрий Рыжиков Dmirtiy
#Андрей Глущенко Andrey GL
#Alexandru Habasescu 
#Дмитрий Репников 
#Родион Тульский 

In [624]:
# Задача на предсказание: согласится ли клиент на депозит.
#Нужно решить задачу с помощью 3х методов:
# 1. кнн
# 2. дерева принятия решения
# 3. логистической регрессии
# и залить, на ваш взгляд, лучшее решение.

In [625]:
# Решение:

In [626]:
# Подключим нужные библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO
from IPython.display import Image

%matplotlib inline
 
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (18,12)

from ipywidgets import interact, IntSlider, FloatSlider

In [627]:
# Подключим файлы с данными
df_sample = pd.read_csv('sample_submission.csv', sep=',', encoding='utf8')
df_test = pd.read_csv('test.csv', sep=',', encoding='utf8')
df_train = pd.read_csv('train.csv', sep=',', encoding='utf8')

In [628]:
# Проверим на пропуски
df_test.info()
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16476 entries, 0 to 16475
Data columns (total 21 columns):
_id               16476 non-null object
age               16476 non-null int64
job               16476 non-null object
marital           16476 non-null object
education         16476 non-null object
default           16476 non-null object
housing           16476 non-null object
loan              16476 non-null object
contact           16476 non-null object
month             16476 non-null object
day_of_week       16476 non-null object
duration          16476 non-null int64
campaign          16476 non-null int64
pdays             16476 non-null int64
previous          16476 non-null int64
poutcome          16476 non-null object
emp.var.rate      16476 non-null float64
cons.price.idx    16476 non-null float64
cons.conf.idx     16476 non-null float64
euribor3m         16476 non-null float64
nr.employed       16476 non-null float64
dtypes: float64(5), int64(5), object(11)
memory usa

In [629]:
# Посмотрим на Данные
df_test.head()

,_id,age,job,marital,education,default,housing,loan,contact,month,...,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,66810d8e6bf2b41c880a7bc6c8a1e295,57,technician,married,high.school,no,no,yes,cellular,may,...,371,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1
1,ccac3879652b08cb8b44c1920fd93afa,55,unknown,married,unknown,unknown,yes,no,telephone,may,...,285,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0
2,fcccab4d7a76f70647f015f2c84c2af8,33,blue-collar,married,basic.9y,no,no,no,cellular,may,...,52,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1
3,ed8399278c30678dab739045fa12b440,36,admin.,married,high.school,no,no,no,telephone,jun,...,355,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1
4,1d4d62ac5cabcb48bac7112813f290cb,27,housemaid,married,high.school,no,yes,no,cellular,jul,...,189,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1


In [630]:
df_train.head()

,_id,age,job,marital,education,default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,target
0,df7489733b004bbe40d3d37b34f82419,54,technician,married,professional.course,unknown,no,no,cellular,aug,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0
1,905a0b9a5456ee962223033473666be3,36,services,single,basic.4y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,02d69c07ec6d68e31d641edd45ce61cd,53,blue-collar,married,basic.4y,no,yes,no,cellular,aug,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,0
3,e492fbe0cccaf67cdb5c0944dbc8a167,34,blue-collar,married,basic.9y,no,yes,yes,telephone,may,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,ac2dc6f5835bdea8e71dd99b65208d07,29,services,single,university.degree,no,yes,no,telephone,aug,...,1,999,0,nonexistent,-1.7,94.027,-38.3,0.890,4991.6,0


In [631]:
# Целевая переменная, нормального вида
#df_train.loc[:, 'target'].value_counts()

In [632]:
# Видно что данные разнородные
# Необходимо их подготовить
# 1. Удалим ненужные признаки (колонки): default, contact, month, day_of_week
# 2. Преобразуем категориальные признаки 
def preproc(df_init):
    df_preproc = df_init.copy()
    
    # Удалили признаки
    df_preproc = df_preproc.drop(['default', 'contact', 'month', 'day_of_week', '_id'], axis=1)
    
    # Замена категориальных признаков
    df_preproc = pd.get_dummies(df_preproc, columns=['job', 'marital', 'education', 
                                     'housing', 'loan', 'campaign', 
                                     'previous', 'poutcome'])
    
    return df_preproc


In [633]:
# Проверяем
df_preproc = df_train.pipe(preproc)
df_preproc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24712 entries, 0 to 24711
Data columns (total 89 columns):
age                              24712 non-null int64
duration                         24712 non-null int64
pdays                            24712 non-null int64
emp.var.rate                     24712 non-null float64
cons.price.idx                   24712 non-null float64
cons.conf.idx                    24712 non-null float64
euribor3m                        24712 non-null float64
nr.employed                      24712 non-null float64
target                           24712 non-null int64
job_admin.                       24712 non-null uint8
job_blue-collar                  24712 non-null uint8
job_entrepreneur                 24712 non-null uint8
job_housemaid                    24712 non-null uint8
job_management                   24712 non-null uint8
job_retired                      24712 non-null uint8
job_self-employed                24712 non-null uint8
job_services     

In [634]:
from sklearn.cross_validation import train_test_split
A_train= df_preproc['target'] # Обучающий столбец
B_train = df_preproc.drop('target', axis=1) # Обучаемый массив данных
# Колонки campaign_42', 'campaign_56', 'previous_7' выкидываем из обучения, 
# т.к. по итогу убучающие и тестовые выборки не сходятся по объему колоной,
# а значит алгоритм будет выпадать в ошибку при предсказании

In [635]:
X_train, X_test, y_train, y_test = train_test_split(B_train, A_train, test_size=0.2, 
                 random_state=123)

In [636]:
# Первый метод - КНН. Сделаем пайплайн
# 1) Нормализатор признаков
# 2) kNN

In [637]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [638]:
model1 = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=7))
])

In [639]:
model1.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=2,
           weights='uniform'))])

In [640]:
y_hat = model1.predict(X_test)

In [641]:
y_hat_proba = model1.predict_proba(X_test, )

In [642]:
from sklearn.metrics import roc_auc_score

In [643]:
roc_auc_score(y_test, y_hat_proba[:, 1])

0.806507134666995

In [644]:
# Второй метод деревья принятия решения
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from ipywidgets import interact, IntSlider
import subprocess

try:
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.model_selection import StratifiedKFold
except ImportError:
    from sklearn.cross_validation import GridSearchCV
    from sklearn.cross_validation import RandomizedSearchCV
    from sklearn.cross_validation import StratifiedKFold

RND_SEED = 123

In [645]:

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': randint(2, 8),
    'min_samples_leaf': randint(5, 10),
    'class_weight': [None, 'balanced']}

cv = StratifiedKFold(n_splits=5, random_state=123, shuffle=True)

model = DecisionTreeClassifier(random_state=123)
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=200, n_jobs=-1,
                                   cv=cv, scoring='roc_auc', random_state=123)

random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=123, shuffle=True),
          error_score='raise',
          estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best'),
          fit_params=None, iid=True, n_iter=200, n_jobs=-1,
          param_distributions={'criterion': ['gini', 'entropy'], 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x124bd8358>, 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x124bd8da0>, 'class_weight': [None, 'balanced']},
          pre_dispatch='2*n_jobs', random_state=123, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=0)

In [646]:
random_search.best_params_

{'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 5,
 'min_samples_leaf': 9}

In [647]:
random_search.best_score_

0.9334295086611445

In [648]:
best_model = random_search.best_estimator_
best_model

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
            max_depth=5, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=9, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')

In [649]:
# Третий метод логистической регрессии
from sklearn.linear_model import LogisticRegression

In [650]:
model2 = LogisticRegression()
model2.fit(X_train, y_train)
predict = model2.predict(X_test)

In [651]:
predict_proba = model2.predict_proba(X_test)
roc_auc_score(y_test, predict_proba[:,1])

0.9256694631609754

In [652]:
# Вычислим предсказание для тестового массива данных
def preproc2(df_init2):
    df_preproc2 = df_init2.copy()
    
    # Удалили признаки
    df_preproc2 = df_preproc2.drop(['default', 'contact', 'month', 'day_of_week', '_id'], axis=1)
    
    # Замена категориальных признаков
    df_preproc2 = pd.get_dummies(df_preproc2, columns=['job', 'marital', 'education', 
                                     'housing', 'loan', 'campaign', 
                                     'previous', 'poutcome'])
    
    return df_preproc2

In [653]:
# Проверяем
df_preproc2 = df_test.pipe(preproc2)
df_preproc2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16476 entries, 0 to 16475
Data columns (total 86 columns):
age                              16476 non-null int64
duration                         16476 non-null int64
pdays                            16476 non-null int64
emp.var.rate                     16476 non-null float64
cons.price.idx                   16476 non-null float64
cons.conf.idx                    16476 non-null float64
euribor3m                        16476 non-null float64
nr.employed                      16476 non-null float64
job_admin.                       16476 non-null uint8
job_blue-collar                  16476 non-null uint8
job_entrepreneur                 16476 non-null uint8
job_housemaid                    16476 non-null uint8
job_management                   16476 non-null uint8
job_retired                      16476 non-null uint8
job_self-employed                16476 non-null uint8
job_services                     16476 non-null uint8
job_student      

In [654]:
df_preproc2.columns # Тестовые данные после подготовки

Index(['age', 'duration', 'pdays', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'job_admin.',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'housing_no', 'housing_unknown', 'housing_yes',
       'loan_no', 'loan_unknown', 'loan_yes', 'campaign_1', 'campaign_2',
       'campaign_3', 'campaign_4', 'campaign_5', 'campaign_6', 'campaign_7',
       'campaign_8', 'campaign_9', 'campaign_10', 'campaign_11', 'campaign_12',
       'campaign_13', 'campaign_14', 'campaign_15', 'campaign_

In [655]:
df_preproc.columns # Обучающие данные после подготовки

Index(['age', 'duration', 'pdays', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'target', 'job_admin.',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'housing_no', 'housing_unknown', 'housing_yes',
       'loan_no', 'loan_unknown', 'loan_yes', 'campaign_1', 'campaign_2',
       'campaign_3', 'campaign_4', 'campaign_5', 'campaign_6', 'campaign_7',
       'campaign_8', 'campaign_9', 'campaign_10', 'campaign_11', 'campaign_12',
       'campaign_13', 'campaign_14', 'campaign_15', 

In [656]:
# Видно, что получили разное число столбцов в тестовой и обучающей выборке после подготовки данных
# При такой ситуации применить обученный алгоритм к тестовым данным мы не можем
# Предлагается оставить в массивах только общие столбцы
# Как следствие уберем из обучющего массива столбцы:
# 'campaign_34', 'campaign_37', campaign_39', 'campaign_41', 'campaign_43',
# А из тестовой столбцы: campaign_42', 'campaign_56', 'previous_7',


In [657]:
# Переобучим нашу модель
from sklearn.cross_validation import train_test_split
A_train2= df_preproc['target'] # Обучающий столбец
B_train2 = df_preproc.drop(['target', 'campaign_34', 
                            'campaign_37', 'campaign_39', 
                            'campaign_41', 'campaign_43'], axis=1) # Обучаемый массив данных
# Колонки campaign_42', 'campaign_56', 'previous_7' выкидываем из обучения, 
# т.к. по итогу убучающие и тестовые выборки не сходятся по объему колоной,
# а значит алгоритм будет выпадать в ошибку при предсказании

In [658]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(B_train2, A_train2, test_size=0.2, 
                 random_state=123)

In [659]:
param_grid2 = {
    'criterion': ['gini', 'entropy'],
    'max_depth': randint(2, 8),
    'min_samples_leaf': randint(5, 15),
    'class_weight': [None, 'balanced']}

cv2 = StratifiedKFold(n_splits=5, random_state=123, shuffle=True)

model2 = DecisionTreeClassifier(random_state=123)
random_search2 = RandomizedSearchCV(model2, param_distributions=param_grid2, n_iter=200, n_jobs=-1,
                                   cv=cv2, scoring='roc_auc', random_state=123)

random_search2.fit(X_train2, y_train2)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=123, shuffle=True),
          error_score='raise',
          estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best'),
          fit_params=None, iid=True, n_iter=200, n_jobs=-1,
          param_distributions={'criterion': ['gini', 'entropy'], 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x11cdf5080>, 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x129b66198>, 'class_weight': [None, 'balanced']},
          pre_dispatch='2*n_jobs', random_state=123, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=0)

In [660]:
random_search2.best_params_

{'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 6,
 'min_samples_leaf': 12}

In [661]:
random_search2.best_score_ 

0.9345715129128995

In [662]:
best_model2 = random_search2.best_estimator_
best_model2

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
            max_depth=6, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=12, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')

In [663]:
df_preproc2 = df_preproc2.drop(['campaign_42', 'campaign_56', 'previous_7'], axis=1)

In [664]:
predict2 = best_model2.predict(df_preproc2)
print(predict2)

[0 0 0 ... 0 0 0]


In [665]:
df = pd.DataFrame(predict2)
df.to_csv("foo.csv")
df

,0
0,0
1,0
2,0
3,0
4,0
5,1
6,0
7,0
8,0
9,0
